<font size=20>Learning regular sets from queries and counterexamples</font>

# Reference paper

## Problem statement

[Learning regular sets from queries and counterexamples](https://www.sciencedirect.com/science/article/pii/0890540187900526)

A `Teacher` knows a regular language $\mathcal{L}(M)$ over an alphabet $\Sigma$ realized by a finite deterministic automaton $M$.

A `Learner` tries to discover $M$. To this end, the `Learner` progressively builds an automaton $H$ based on the answers obtained from `Teacher`. The `Learner` can ask two types of questions:
* Do $H$ and $M$ corresponds?
  * ... where $H$ denotes the current hypothese automaton of the `Learner`.
  * If not the teacher provides a counter example, i.e. a word $w \in (L \backslash \mathcal{L}(H)) \cup (\mathcal{L}(H) \backslash \mathcal{L}(M))$
* Does $w$ belong to $\mathcal{L}(M)$?

## $L^*$ algorithm 

We denote by the empty word by $\varepsilon$ (N.B. In her article, D. Angluin uses $\lambda$ instead).

The `Learner` maintains a triple $(S, E, T)$:
* $S$ is a set of prefixes over $\Sigma^*$, initialized to $\{\varepsilon\}$;
* $E$ is a set of suffixes over $\Sigma^*$, initialized to $\{\varepsilon\}$;
* $T$ is the `ObservationTable`, which is in practice a 0-1 matrix $T$.
  * Each row $i$ corresponds to a given prefix $s \in S \cup S.\Sigma$, where $S.\Sigma = {s.a, s \in S, a \in \Sigma}$;
  * Each column $j$ corresponds to a given suffix $e \in E$;
  * $T(i,j)$ indicates whether the word $s.e$ belongs to $L$ or not.

The `Learner` updates progressively its $(S,E,T)$ triple thanks to the $L^*$ algorithm. It derives the hypothese automaton $H$ from $T$. This work is made in two steps:

__Exploration:__

To build the hypothese minimal complete DFA $H$, the `Learner` requires a _closed_ and _consistent_ `ObservationTable` (these properties are defined later). To this end, the `Learner` triggers membership queries to the `Teacher`, allowing to extend $(S, E, T)$ until these two properties hold.

Once $(S,E,T)$ is closed and consistent, the `Learner` derives $H$ as follows:
* _States:_ Each row of $S$ identifies a state of $H$. This $q$ state is when $H$ reads the prefix $s \in S$ matching this row. $q$ is final iff $T(s, \varepsilon)$ is true.
* _Transitions:_ To determine egress transition of $q$, for all $a \in \Sigma$, the algorithm searches the row corresponding to $s+a$. This row identifies a prefix of $S$ and thus exactly one state $r$ of $H$. Thus, the $a$-transition from $q$ to $r$ is built.

At the end, the `Learner` obtained a minimal finite deterministic automaton $H$.

__Validation:__

The `Learner` proposes $H$ the `Teacher`.
* If the `Teacher` does not return counter example, it means that $H$ is the expected automaton.
* Otherwise, the `Teacher` returns counter-example $t$. The `Learner` insert $t$ and all its prefixes into $S$. By doing so, the `Learner` is guaranteed to only propose in the future automata returning the right result for the word $t$. Then, the `Learner` repeats the Exploration and Validation phases until $H = M$.

# Deterministic Finite Automaton (DFA)
## Implementation

In [ ]:
from pybgl.automaton import Automaton, accepts, add_edge, alphabet, delta, edge, \
    finals, initial, is_complete, is_deterministic, is_final, is_finite, is_initial, \
    is_minimal, label, make_automaton, set_final, sigma

# Teacher

The teacher is just a wrapper around a (minimal) deterministic automaton $M$.

It exposes two main primitives:
* _membership queries:_ "$w \in \mathcal{L}(M)$?" for some word $w \in \Sigma^*$.
* _equivalence queries:_ "$\mathcal{L}(H) = \mathcal{L}(M)$?" for some hypothesis automaton $H$.


In [ ]:
from lstar.teacher import Teacher

## Membership queries

It just consists in testing whether $w$ is accepted by the automaton $M$.

## Equivalence queries

The goal here is to determine whether two automata $G_1$ and $G_2$ recognize the same language, i.e. if $\mathcal{L}(G_1)$ = $\mathcal{L}(G_2)$.
* __Method 1:__ build in polynomial time the automaton recognizing $(\mathcal{L}(G_1) \backslash \mathcal{L}(G_2)) \cup (\mathcal{L}(G_2) \backslash \mathcal{L}(G_1))$, and then extract any accepted word by the resulting automaton. 
* __Method 2:__ build the product automaton $G_1 \times G_2$. If a pair $(q_1,q_2) \in (Q_1 \times \{\perp\}) \times (\{\perp\} \times Q_2)$ is discovered then a contradiction has been stop and we can stop the exploration.

In the $L^*$ algorithm, we can simplify this test. Indeed $H$ is complete and minimal. By assuming that $M$ is also minimal, our implementation `automaton_match` only needs to check if there is bijection between the states of $M$ and $H$.

In [ ]:
from lstar.automaton_match import automaton_match

# Learner

## Observation table $(S,E,T)$

An `ObservationTable` is a matrix in ${0,1}^{|S \cup S.\Sigma| \times |E|}$ used to build the hypothesis automaton $H$.
* We call _signature_ the row vector in ${0,1}^{|E|}$ related to a suffix $s$ in $S \cup S.\Sigma$. We denote it by $\mathrm{row}(s)$.
* Each signature of $s \in S$ identifies exactly one state in the inferred automaton $H$.
* Each row corresponding to a suffix in $S.\Sigma$ helps to build complete automaton.

__Definitions:__
* An `ObservationTable` is said to be _complete_ (or _closed_) iff $\forall t \in S.A, \exists s \in S~|~row(t) = row(s)$.
* An `ObservationTable` is said to be _consistent_ (or _separable_) iff $\forall s \in S, \forall s' \in S, \forall a \in \Sigma~|~row(s) = row(s) \implies row(s.a) = row(s'.a)$.

In [ ]:
from lstar.observation_table import ObservationTable

## Building the hypothesis automata $H$ from $(S,E,T)$

If an `ObservationTable` is _complete_, _consistent_, then the `Learner` can derive a deterministic automaton $H$ and submit it to the `Teacher`.
* __States:__ each state $q$ of $H$ is identified by a prefix $s \in S$ (in particular $\varepsilon$ identifies the initial state). Each state is identified $\mathrm{row}(s)$. _Separability_ guarantees that these states are distinguishable
* __Transitions:__  for each state $q$ related to a suffix $s$, and for all $a \in \Sigma$, the `Learner` can examine in $T$ the signature $\mathrm{row}(s.a)$. This identifies a unique node $q'$ because $T$ is _separable_ and _closed_. This results to a $a$-transition from $q$ to $q'$.

Once $H$ is built, the `Learner` can submit an equivalence query to the `Teacher` to test whether $H$ and $M$ match. If the `Teacher` returns a counter-example, the `Learner` updates its `ObservationTable` $T$. In particular, the `Learner` needs to fill $T$ and extend it until $T$ becomes complete and consistent. This forces the `Learner` to trigger several new membership queries.

In [ ]:
from lstar.learner import Learner, make_automaton_from_observation_table

# Tests

In [ ]:
from pybgl.graph import GraphvizStyle

# Uncomment the following instruction for notebook using dark theme
GraphvizStyle.set_fg_color("grey")

In [ ]:
from pybgl.automaton            import Automaton, is_complete, make_automaton, vertices
from pybgl.graphviz             import dotstr_to_html
from pybgl.html                 import html
from pybgl.property_map         import make_func_property_map
from lstar.automaton_match      import automaton_match
from lstar.observation_table    import ObservationTable
from lstar.learner              import Learner, make_automaton_from_observation_table
from lstar.teacher              import Teacher

G1 = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
        (1, 2, 'a'), (1, 1, 'b'),
        (2, 1, 'a'), (2, 1, 'b'),
    ], 0, 
    make_func_property_map(lambda q : q in {1})
)

G2 = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
    ], 0,
    make_func_property_map(lambda q : q in {1})
)

G3 = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
    ], 0,
    make_func_property_map(lambda q : False)
)

G4 = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
        (1, 1, 'b'), (1, 0, 'a')
    ], 0,
    make_func_property_map(lambda q : q in {1})
)

G5 = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
        (1, 1, 'b'), (1, 0, 'a')
    ], 0,
    make_func_property_map(lambda q : False)
)

def test_learner(g :Automaton, verbose :bool = False, write_files :bool = False):
    if not is_complete(g):
        html(dotstr_to_html(g.to_dot()))
        html("Ignored, this automaton must be finite, deterministic and complete")
        return

    teacher = Teacher(g)
    html("<b>Teacher</b>")
    html(dotstr_to_html(teacher.g.to_dot()))

    learner = Learner(teacher)
    h = learner.learn(verbose = verbose, write_files = write_files)
    html("<b>Learner</b>")
    html(dotstr_to_html(h.to_dot()))

    assert automaton_match(g, h) == None
    html(":-)")

def test_learners(gs = [G1, G2, G3, G4, G5], verbose = False):
    for (i, g) in enumerate(gs):
        html("<h3>Test G%d</h3>" % (i + 1))
        test_learner(g, verbose)
        
test_learners(verbose = True)

In [ ]:
G_DEMO = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
        (1, 2, 'a'), (1, 1, 'b'),
        (2, 1, 'a'), (2, 1, 'b'),
    ], 0, make_func_property_map(lambda q: q in {1})
)

test_learner(G_DEMO, verbose=True)

# Sandbox (for slides)

In [ ]:
import os
from pybgl.graphviz import dotstr_to_html
from pybgl.html     import html

PREFIX = "./"

def write_automaton(g :Automaton, filename :str):
    svg = dotstr_to_html(g.to_dot())
    html(svg)
    with open(filename, "w") as f:
        print("Writting [%s]" % filename)
        print(svg, file=f)
    
write_automaton(G1, os.path.join(PREFIX, "out_automaton.svg"))

In [ ]:
SMALL_DFA = make_automaton(
    [
        (0, 0, 'a'), (0, 1, 'b'),
        (1, 1, 'b'), (1, 0, 'a')
    ], 0, make_func_property_map(lambda q: q in {1})
)

write_automaton(SMALL_DFA, os.path.join(PREFIX, "small_dfa.svg"))

In [ ]:
SMALL_TRIE = make_automaton(
    [
        (0, 1, 'a'), (1, 2, 'b'), (2, 3, 'a'), (2, 4, 'b'),
        (1, 5, 'x')
    ], 0, make_func_property_map(lambda q: q in {2,3,4,5})
)

write_automaton(SMALL_TRIE, os.path.join(PREFIX, "small_trie.svg"))